In [1]:
from firebase_admin import credentials, firestore, initialize_app
from google.cloud import storage
import requests
import time

In [2]:
cred = credentials.Certificate("./firebase_key.json")
app = initialize_app(cred)
db = firestore.client()

storage_client = storage.Client()
bucket = storage_client.get_bucket("control-meme-public")


In [9]:


def add_variation_to_data(image_path_local, parent_uuid, **kwargs):
    
    # get kwargs
    prompt = kwargs.get("prompt")
    controlnetPreprocess = kwargs.get("controlnetPreprocess")
    controlnetModel = kwargs.get("controlnetModel")
    baseModel = kwargs.get("baseModel")
    nb_steps = kwargs.get("nb_steps")
    sampler = kwargs.get("sampler")
    seed = kwargs.get("seed")
    guidance_strenght_prompt = kwargs.get("guidance_strenght_prompt")
    guidance_strenght_image = kwargs.get("guidance_strenght_image")
    
    # save meme variation to bucket
    
    formated_timestamp = time.strftime("%Y%m%d-%H%M%S")
    bucket_save_path = "meme_variation_" + formated_timestamp + ".jpeg"
    blob = bucket.blob(bucket_save_path)
    blob.upload_from_filename(image_path_local, content_type="image/jpeg")
    url = "https://storage.googleapis.com/control-meme-public/" + bucket_save_path
    
    # save meme variation to firestore as a child of the meme with id memeID
    db = firestore.client()
    doc_ref = db.collection("BaseMemes").document(parent_uuid).collection("Variations").document()
    doc_ref.set(
        {
            "url": url,
            "prompt": prompt,
            "controlnetPreprocess": controlnetPreprocess,
            "controlnetModel": controlnetModel,
            "baseModel": baseModel,
            "nb_steps": nb_steps,
            "sampler": sampler,
            "seed": seed,
            "guidance_strenght_prompt": guidance_strenght_prompt,
            "guidance_strenght_image": guidance_strenght_image,
            "timestamp": firestore.SERVER_TIMESTAMP,
            "parent_uuid": parent_uuid
        }
    )

In [10]:
import base64
from PIL import Image
import random
import os
with open("./donkey.jpg", "rb") as image_file:
    base64_image = base64.b64encode(image_file.read())
    random_name = str(random.randint(0, 10000))
#save to png
path = f"./{random_name}.jpeg"
with open(path, 'wb') as f:
    f.write(base64.b64decode(base64_image))

add_variation_to_data(path, "0wArdCq9gKcSHlG69Ubx", prompt="test")

os.remove(path)



b'...'


In [5]:
print(base64_image)

b'...'


In [11]:
im = Image.open('./donkey.jpg')

In [21]:
from io import BytesIO

buffered = BytesIO()
im.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue())


In [24]:
data

{'imageb64': b'...',
 'prompt': 'cute donkey'}

In [35]:
data = dict(
    memeID = "KDGtOH37Lv9J3JPUcMuM",
    imageb64=img_str.decode("utf-8"),
    prompt="cute donkey"
)
requests.post('http://localhost:5000/api/save_variation/', json = data)


In [36]:
data

{'memeID': 'KDGtOH37Lv9J3JPUcMuM',
 'imageb64': '...',
 'prompt': 'cute donkey'}

In [41]:
help(requests.post)

Help on function post in module requests.api:

post(url, data=None, json=None, **kwargs)
    Sends a POST request.
    
    :param url: URL for the new :class:`Request` object.
    :param data: (optional) Dictionary, list of tuples, bytes, or file-like
        object to send in the body of the :class:`Request`.
    :param json: (optional) json data to send in the body of the :class:`Request`.
    :param \*\*kwargs: Optional arguments that ``request`` takes.
    :return: :class:`Response <Response>` object
    :rtype: requests.Response



In [43]:

requests.post('http://localhost:5000/api/save_variation/', json = data)

<Response [200]>